In [1]:
import requests
import json
import numpy as np
import pandas as pd
import csv
import time
from math import sin, cos,asin, degrees,sqrt, atan2, radians
from datetime import datetime
import shapely.geometry 

## 4.5.1 Análisis de las fases

In [2]:
bases_fases = pd.read_csv('bases_bicimad_fases.csv', sep=";", header=0,encoding= 'unicode_escape')

In [4]:
bases_api = pd.read_csv('bicimad_stations.csv', sep=";", header=0)

In [3]:
bases_fases.sample(3)

,Número,Gis_X,Gis_Y,Fecha de Alta,Distrito,Barrio,Calle,Nº Finca,Tipo de Reserva,Número de Plazas,Longitud,Latitud,Direccion
167,163,"440055,04","4472797,85",01/07/2015,02 ARGANZUELA,02-02 ACACIAS,"ESPERANZA, PASEO, DE LA",2 A,BiciMAD,21,-3.706437,40.403635,"ESPERANZA, PASEO, DE LA, 2 A"
66,68,"441358,92","4473802,81",23/06/2014,03 RETIRO,03-05 JERÓNIMOS,"ESPALTER, CALLE, DE",1,BiciMAD,21,-3.691165,40.412781,"ESPALTER, CALLE, DE, 1"
125,123,"439482,59","4475800,04",23/06/2014,07 CHAMBERÍ,07-01 GAZTAMBIDE,"GUZMAN EL BUENO, CALLE, DE",2,BiciMAD,24,-3.713468,40.430639,"GUZMAN EL BUENO, CALLE, DE, 2"


In [10]:
bases_api.sample(5)

,id,number,name,address,activate,no_available,total_bases,longitude,latitude
186,192,184,Marqués de Zafra,"Pº Marqués de Zafra, 24",1,1,24,-3.665306,40.426000
177,183,175,Jaime el Conquistador,"Jaime I el Conquistador, 30",1,1,21,-3.698306,40.396222
4,5,4,Malasaña,Calle Manuela Malasaña nº 5,1,1,24,-3.702587,40.428552
46,50,46,Ribera de Curtidores,Calle Ribera de Curtidores nº 28,1,1,24,-3.707126,40.405315
205,211,203,San Francisco de Sales,"General Ampudia, 2",1,1,24,-3.714472,40.441750


##### Número de estaciones de bicis del fichero 'bases_bicimad.csv'

In [7]:
bases_fases.count()[0]

216

##### Número de estaciones de bicis de la API EMT

In [12]:
bases_api.count()[0]

214

#### Número de estaciones por año

In [61]:
bases_fases.groupby('fecha_alta_year').size()

fecha_alta_year
2014    122
2015     41
2016      1
2017      2
2018      4
2019     46
dtype: int64

In [14]:
bases_fases_new = bases_fases[['Número','Fecha de Alta']]
bases_fases_new.columns = ['number', 'fecha_alta']
bases_fases_new.head(5)

,number,fecha_alta
0,001 a,23/06/2014
1,001 b,23/06/2014
2,2,23/06/2014
3,3,23/06/2014
4,4,23/06/2014


In [15]:
result = pd.merge(bases_api, bases_fases_new, how='left', on=['number'],indicator='indicator_column')

##### Número de estaciones de bicis de la API EMT no encontradas en el conjunto de datos 'bases_bicimad.csv'

In [16]:
result[result['indicator_column'] == 'left_only'] 

,id,number,name,address,activate,no_available,total_bases,longitude,latitude,fecha_alta,indicator_column
0,1,1a,Puerta del Sol A,Puerta del Sol nº 1,1,1,30,-3.701781,40.417153,NaN,left_only
1,2,1b,Puerta del Sol B,Puerta del Sol nº 1,1,1,30,-3.702421,40.417001,NaN,left_only
20,21,20a,Banco de España A,Calle Alcalá nº 49,1,1,30,-3.695461,40.419234,NaN,left_only
24,28,25a,Plaza de Celenque A,Plaza de Celenque nº 1,1,1,24,-3.706481,40.417311,NaN,left_only
25,29,25b,Plaza de Celenque B,Plaza de Celenque nº 1,1,1,24,-3.706384,40.417278,NaN,left_only
28,32,28,Sevilla,Calle Alcalá nº 27,1,1,24,-3.699260,40.418166,NaN,left_only
78,84,80a,Atocha A,Paseo de la Infanta Isabel nº 3,1,1,24,-3.690225,40.407568,NaN,left_only
79,85,80b,Atocha B,Paseo de la Infanta Isabel nº 3,1,1,27,-3.690123,40.407490,NaN,left_only
105,111,106a,Colón A,Calle Serrano nº 34,1,1,18,-3.687723,40.425100,NaN,left_only
106,112,106b,Colón B,Calle Serrano nº 34,1,1,18,-3.687745,40.424963,NaN,left_only


In [28]:
result[result['indicator_column'] == 'left_only'] .count()[0]

15

##### Lectura del fichero con la asignación manual de claves

In [5]:
bases_fases = pd.read_csv('bases_bicimad_update.csv', sep=";", header=0)

In [6]:
bases_fases['Fecha de Alta'] = pd.to_datetime(bases_fases['Fecha de Alta'],format="%d/%m/%Y")

In [7]:
bases_fases['fecha_alta_year'] = bases_fases['Fecha de Alta'].dt.year

In [8]:
def set_etapa(year):
    if(year == 2014):
        return "1"
    elif (year == 2015):
        return "2"
    elif (year == 2019):
        return "3"
    else:
        return "-1"

bases_fases['etapa'] = bases_fases.apply(lambda x: set_etapa(x.fecha_alta_year), axis=1)

In [9]:
bases_fases_new = bases_fases[['Número','Fecha de Alta','Calle','Distrito','Barrio','etapa']]
bases_fases_new.columns = ['number', 'fecha_alta','direccion','distrito','barrio','etapa']

In [10]:
estaciones_bici_etapa = pd.merge(bases_api, bases_fases_new, how='left', on=['number'],indicator='indicator_column')

##### Estaciones de bicicleta sin clasificar

In [11]:
estaciones_bici_etapa[(estaciones_bici_etapa['indicator_column'] == 'left_only') | (estaciones_bici_etapa['etapa'] == "-1")] 

,id,number,name,address,activate,no_available,total_bases,longitude,latitude,fecha_alta,direccion,distrito,barrio,etapa,indicator_column
20,21,20a,Banco de España A,Calle Alcalá nº 49,1,1,30,-3.695461,40.419234,2018-07-10,"ALCALA, CALLE, DE",01 CENTRO,01-04 JUSTICIA,-1,both
28,32,28,Sevilla,Calle Alcalá nº 27,1,1,24,-3.699260,40.418166,NaT,NaN,NaN,NaN,NaN,left_only
105,111,106a,Colón A,Calle Serrano nº 34,1,1,18,-3.687723,40.425100,NaT,NaN,NaN,NaN,NaN,left_only
106,112,106b,Colón B,Calle Serrano nº 34,1,1,18,-3.687745,40.424963,NaT,NaN,NaN,NaN,NaN,left_only
132,157,130,Santa Engracia 127,Calle Santa Engracia nº 127,1,1,20,-3.701632,40.441386,2017-06-21,"SANTA ENGRACIA, CALLE, DE",07 CHAMBERÍ,07-05 RÍOS ROSAS,-1,both
168,174,166,Segovia 26,"Calle Segovia, 26",1,1,24,-3.713583,40.413833,2016-06-23,"SEGOVIA, CALLE, DE",01 CENTRO,01-01 PALACIO,-1,both
169,175,167,Segovia 45,"Calle Segovia, 45",1,1,24,-3.717487,40.413736,2017-05-22,"SEGOVIA, CALLE, DE",02 ARGANZUELA,02-01 IMPERIAL,-1,both
180,186,178,Junta Municipal Retiro,Plaza de Daoíz y Velarde,1,1,24,-3.678834,40.402920,NaT,NaN,NaN,NaN,NaN,left_only


In [243]:
estaciones_bici_etapa.to_csv('estaciones_etapa_emt.csv', sep=';', encoding='utf-8',index=False)

In [15]:
estaciones_bici_etapa.head(5)

,id,number,name,address,activate,no_available,total_bases,longitude,latitude,fecha_alta,direccion,distrito,barrio,etapa,indicator_column
0,1,1a,Puerta del Sol A,Puerta del Sol nº 1,1,1,30,-3.701781,40.417153,2014-06-23,"PUERTA DEL SOL, PLAZA, DE LA",01 CENTRO,01-06 SOL,1.0,both
1,2,1b,Puerta del Sol B,Puerta del Sol nº 1,1,1,30,-3.702421,40.417001,2014-06-23,"ALCALA, CALLE, DE",01 CENTRO,01-06 SOL,1.0,both
2,3,2,Miguel Moya,Calle Miguel Moya nº 1,1,1,24,-3.705842,40.420589,2014-06-23,"MIGUEL MOYA, CALLE, DE",01 CENTRO,01-05 UNIVERSIDAD,1.0,both
3,4,3,Plaza Conde Suchil,Plaza del Conde Suchil nº 2-4,1,1,18,-3.706917,40.430294,2014-06-23,"CONDE DEL VALLE DE SUCHIL, PLAZA, DEL",07 CHAMBERÍ,07-02 ARAPILES,1.0,both
4,5,4,Malasaña,Calle Manuela Malasaña nº 5,1,1,24,-3.702587,40.428552,2014-06-23,"MANUELA MALASAÑA, CALLE, DE",01 CENTRO,01-05 UNIVERSIDAD,1.0,both


##### Análisis con los movimientos

In [18]:
estaciones_bicimad_totales = pd.read_csv('estaciones_bicimad_totales_agregado.csv', sep=";", header=0)

In [19]:
movimientos = pd.read_csv('movimientos_general.csv', sep=";", header=0)

In [20]:
lines_movimientos = [key for key, data in movimientos.groupby('id_line')]

In [21]:
lines_totales = [key for key, data in estaciones_bicimad_totales.groupby('id_line')]

In [22]:
lines_not = list(set(lines_totales)-set(lines_movimientos))

##### Líneas no encontradas en los movimientos

In [23]:
lines_not

[1, 37, 12, 148, 149, 152, 56, 27]

##### Extracción de las rutas de bicicleta en función de las líneas donde no se encontraron movimientos

In [24]:
rutas_not = estaciones_bicimad_totales[estaciones_bicimad_totales['id_line'].isin(lines_not)][['id_line','id_header_origen','name_header_origen','id_bicimad_origen','id_header_destino','name_header_destino','id_bicimad_destino']]

In [25]:
estaciones_bici_etapa = pd.read_csv('estaciones_etapa_emt.csv', sep=";", header=0)
estaciones_bici_etapa_1 = estaciones_bici_etapa[['id','etapa','fecha_alta']]
estaciones_bici_etapa_1['id_bicimad_origen'] = estaciones_bici_etapa_1['id']
estaciones_bici_etapa_1.drop('id', axis=1, inplace=True)
aux = pd.merge(rutas_not, estaciones_bici_etapa_1, how='left', on=['id_bicimad_origen'])
aux.rename(columns={'etapa': 'etapa_origen', 'fecha_alta': 'fecha_alta_origen'}, inplace=True)
estaciones_bici_etapa_1['id_bicimad_destino'] = estaciones_bici_etapa_1['id_bicimad_origen']
estaciones_bici_etapa_1.drop('id_bicimad_origen', axis=1, inplace=True)
aux = pd.merge(aux, estaciones_bici_etapa_1, how='left', on=['id_bicimad_destino'])
aux.rename(columns={'etapa': 'etapa_destino', 'fecha_alta': 'fecha_alta_destino'}, inplace=True)
aux

C:\Users\Arturo\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\Arturo\Anaconda3\lib\site-packages\pandas\core\frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
C:\Users\Arturo\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the d

,id_line,id_header_origen,name_header_origen,id_bicimad_origen,id_header_destino,name_header_destino,id_bicimad_destino,etapa_origen,fecha_alta_origen,etapa_destino,fecha_alta_destino
0,1,4514,Cristo Rey,160,273,Prosperidad,204,2.0,2015-07-01,3.0,2019-10-21
1,12,4541,Cristo Rey,160,779,Marqués de Zafra,193,2.0,2015-07-01,3.0,2019-08-01
2,12,4541,Cristo Rey,211,779,Marqués de Zafra,193,3.0,2019-05-27,3.0,2019-08-01
3,27,86,Embajadores,51,5602,Plaza Castilla,216,1.0,2014-06-23,3.0,2019-09-30
4,27,86,Embajadores,51,5602,Plaza Castilla,217,1.0,2014-06-23,3.0,2019-09-30
5,27,86,Embajadores,52,5602,Plaza Castilla,216,1.0,2014-06-23,3.0,2019-09-30
6,27,86,Embajadores,52,5602,Plaza Castilla,217,1.0,2014-06-23,3.0,2019-09-30
7,27,86,Embajadores,53,5602,Plaza Castilla,216,1.0,2014-06-23,3.0,2019-09-30
8,27,86,Embajadores,53,5602,Plaza Castilla,217,1.0,2014-06-23,3.0,2019-09-30
9,37,1381,Cuatro Caminos,149,1000,Puente de Vallecas,187,2.0,2015-07-01,3.0,2019-09-30


##### Número de rutas sin movimientos

In [26]:
rutas_not.count()[0]

31

##### Número de rutas con alguna cabecera de origen inaugurada en la tercera fase

In [64]:
aux[aux['etapa_origen'] == 3].count()[0]

4

##### Número de rutas con alguna cabecera de destino inaugurada en la tercera fase

In [65]:
aux[aux['etapa_destino'] == 3].count()[0]

31

##### Fecha más antigua de inauguración de alguna cabecera de destino

In [71]:
aux[aux['etapa_destino'] == 3][['fecha_alta_destino']].min()[0]

'2019-08-01'

##### Fecha más reciente de inauguración de alguna cabecera de destino

In [70]:
aux[aux['etapa_destino'] == 3][['fecha_alta_destino']].max()[0]

'2019-10-21'

## 4.5.2 Asignación de estaciones aledañas

#### Seleccionamos aquellas estaciones de origen distintas

In [73]:
list_origen = aux[aux['etapa_origen'] == 3].drop_duplicates('id_bicimad_origen')[['id_bicimad_origen']].sort_values('id_bicimad_origen')['id_bicimad_origen'].to_list()
origen_fase_iii = bases_api[bases_api['id'].isin(list_origen)][['id','latitude','longitude']]
origen_fase_iii

,id,latitude,longitude
193,199,40.434639,-3.674000
196,202,40.437250,-3.677222
205,211,40.441750,-3.714472
206,212,40.447667,-3.704167


#### Seleccionamos aquellas estaciones de destino distintas

In [28]:
list_destino = aux[aux['etapa_destino'] == 3].drop_duplicates('id_bicimad_destino')[['id_bicimad_destino']].sort_values('id_bicimad_destino')['id_bicimad_destino'].to_list()
destino_fase_iii = bases_api[bases_api['id'].isin(list_destino)][['id','latitude','longitude']]
destino_fase_iii

,id,latitude,longitude
181,187,40.397972,-3.669250
182,188,40.394139,-3.675889
187,193,40.425083,-3.661528
198,204,40.444139,-3.675139
210,216,40.466361,-3.688639
211,217,40.465917,-3.688472


#### buscamos aquellas estaciones cercanas que no sean de fase III

In [34]:
def calcular_distancia(lat1, lon1, lat2, lon2):
    R = 6373.0
    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * (sin(dlon/2))**2
    c = 2 * atan2( sqrt(a), sqrt(1-a) )
    return R * c * 1000

In [65]:
def calcular_distancia_con_coordinates(row_orig,radio,df):
    res = []
    for index, row in df.iterrows():
        distancia = calcular_distancia(row_orig.latitude,row_orig.longitude,row.latitude,row.longitude) 
        if row.id != row_orig.id and row.etapa != 3 and distancia <= radio:
            row['estacion_ori_id'] = row_orig.id
            res.append(row)
    return pd.DataFrame(res) 

In [66]:
def buscar_estaciones(df,radio):
    aux = []
    for index, row in df.iterrows():
        res = calcular_distancia_con_coordinates(row,radio,estaciones_bici_etapa)
        if(not res.empty): aux.append(res)
    if not aux: return None
    else: return pd.concat(aux)

##### Estaciones que sustituyen a las anteriores en cabecera de origen de bicimad

In [82]:
aledanias_origen = buscar_estaciones(origen_fase_iii,300)[["id","name","fecha_alta","etapa","estacion_ori_id"]]
aledanias_origen

,id,name,fecha_alta,etapa,estacion_ori_id
146,170,Juan Bravo 50,2015-06-20,2.0,199.0
134,149,Glorieta de los Cuatro Caminos,2015-07-01,2.0,212.0


##### Estaciones que sustituyen a las anteriores en cabecera de destino de bicimad

In [80]:
aledanias_destino = buscar_estaciones(destino_fase_iii,300)
aledanias_destino

##### Comprobar si las estaciones aledañas ya han sido agregadas

In [98]:
def comprobar_estaciones_aledanias():
    aux = []
    for index, row in aledanias_origen.iterrows():
        id_line_original = estaciones_bicimad_totales[estaciones_bicimad_totales['id_bicimad_origen'] == row.estacion_ori_id][['id_line']].values[0][0]
        if id_line_original != estaciones_bicimad_totales[estaciones_bicimad_totales['id_bicimad_origen'] == row.id][['id_line']].values[0][0]:
            aux.append(row)
    return aux

In [97]:
print(comprobar_estaciones_aledanias())

[]
